In [247]:
import requests
import pandas as pd
from datetime import datetime

# Some parameters
token_data = {'token': 'bJHNCRwspvRYRMOHApDLoAKBmLRNtdjY'} # token for datasets extraction

In [43]:
# No longer needed reprecated below
#x = pd.read_csv('https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&stations=ASN00084027&startDate=2016-01-01&endDate=2016-01-02&boundingBox=90,-180,-90,180')
#https://www.ncei.noaa.gov/access/services/data/v1?dataset=global-marine&dataTypes=WIND_DIR,WIND_SPEED&stations=AUCE&startDate=2016-01-01&endDate=2016-01-02&boundingBox=90,-180,-90,180
#https://www.ncei.noaa.gov/access/services/data/v1?dataset=global-summary-of-the-year&dataTypes=DP01,DP05,DP10,DSND,DSNW,DT00,DT32,DX32,DX70,DX90,SNOW,PRCP&stations=ASN00084027&startDate=1952-01-01&endDate=1970-12-31&includeAttributes=true&format=pdf
#dt = pd.read_json("../../../tzava/out.json")

#dt["name"]

# Dataset creation 

We want to create a dataset with monthly temperature measurements for several countries. 

## Create a list with all the countries

For the specific category of data, e.g. temperature, we export a list with the countries and their ids where measurements are available.

In [44]:
parameters = {'datasetid':'GSOM', 'locationcategoryid':'CNTRY', 'datacategoryid':'TEMP','limit':500}
x = requests.get('https://www.ncei.noaa.gov/cdo-web/api/v2/locations?', params = parameters, 
                 headers=token_data)
print(x.url)

j = x.json()
j = j['results']

200

{'Date': 'Mon, 24 Jul 2023 05:19:09 GMT', 'Server': 'Apache', 'Strict-Transport-Security': 'max-age=31536000', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': 'GET', 'Access-Control-Allow-Headers': 'token', 'Content-Type': 'application/json', 'Connection': 'close', 'Transfer-Encoding': 'chunked'}
{"metadata":{"resultset":{"offset":1,"count":196,"limit":500}},"results":[{"mindate":"1949-01-01","maxdate":"1969-06-01","name":"Antigua & Barbuda","datacoverage":1,"id":"FIPS:AC"},{"mindate":"1944-04-01","maxdate":"2023-06-01","name":"United Arab Emirates","datacoverage":0.9819,"id":"FIPS:AE"},{"mindate":"1898-09-01","maxdate":"2023-06-01","name":"Afghanistan","datacoverage":1,"id":"FIPS:AF"},{"mindate":"1852-01-01","maxdate":"2023-06-01","name":"Algeria","datacoverage":1,"id":"FIPS:AG"},{"mindate":"1881-08-01","maxdate":"2023-06-01","name":"Azerbaijan","datacoverage":1,"id":"FIPS:AJ"},{"mindate":"1940-04-01","maxdate":"2020-10-01","name":"Albania","datacoverage":0.99

In [46]:
countries_table = pd.DataFrame(j)
countries_table[['name', 'id']]

,name,id
0,Antigua & Barbuda,FIPS:AC
1,United Arab Emirates,FIPS:AE
2,Afghanistan,FIPS:AF
3,Algeria,FIPS:AG
4,Azerbaijan,FIPS:AJ
...,...,...
191,Vietnam,FIPS:VM
192,Namibia,FIPS:WA
193,Swaziland,FIPS:WZ
194,Zambia,FIPS:ZA


In [47]:
# for each of the countries in the list export the temperature information
countries_select = ["France", "Greece"]
# Get the corresponding ids
countries_select_ids = list(countries_table.loc[countries_table['name'].isin(countries_select),'id'])

In [48]:
countries_select_ids

['FIPS:FR', 'FIPS:GR']

## Selection of the categories of features we want to include.
We check for the categories of variables that are available in the dataset that we want to query. We can then use these categories to include the features that we want to study in our dataset.

In [ ]:
# Get the data categories that are available in the dataset
datacategories_parameters = {'datasetid':'GSOM', 'limit':500}
x = requests.get('https://www.ncei.noaa.gov/cdo-web/api/v2/datacategories?', params = datacategories_parameters , 
                 headers=token_data)
print(x.status_code)
j = x.json()
j = j['results']

datacategories_table = pd.DataFrame(j)

200


In [ ]:
print(datacategories_table)

              name    id
0         Computed  COMP
1      Evaporation  EVAP
2             Land  LAND
3    Precipitation  PRCP
4         Sunshine   SUN
5  Air Temperature  TEMP
6             Wind  WIND


For example, we want to use the measurements of temperature and precipitation and explore the relation between those two.

For every category, we want to get the list of variables available in the dataset.

In [221]:
# Sample code used for testing the variables in the dataset
variables_df = pd.DataFrame()
categories = ['TEMP','PRCP']
for category in categories :
    test_parameters = {'datasetid':'GSOM', 'datacategoryid':category,'limit':500}
    response = requests.get('https://www.ncei.noaa.gov/cdo-web/api/v2/datatypes?', params = test_parameters, 
                 headers=token_data)
    print(response.url)
    
    response_json = response.json()
    response_json = response_json['results']

    df = pd.DataFrame(response_json)
    df['datacategoryid']=category

    variables_df = pd.concat([variables_df,df], axis=0)

print(variables_df)

https://www.ncei.noaa.gov/cdo-web/api/v2/datatypes?datasetid=GSOM&datacategoryid=TEMP&limit=500
https://www.ncei.noaa.gov/cdo-web/api/v2/datatypes?datasetid=GSOM&datacategoryid=PRCP&limit=500
      mindate     maxdate                                               name  \
0  1763-01-01  2023-06-01                 Cooling Degree Days Season to Date   
1  1763-01-01  2023-06-01        Extreme minimum temperature for the period.   
2  1763-01-01  2023-06-01        Extreme maximum temperature for the period.   
3  1763-07-01  2023-06-01                 Heating Degree Days Season to Date   
4  1763-01-01  2023-06-01                               Average Temperature.   
5  1763-01-01  2023-06-01                                Maximum temperature   
6  1763-01-01  2023-06-01                                Minimum temperature   
0  1863-12-01  2023-06-01  Number days with snow depth > 1 inch(25.4mm) f...   
1  1840-05-01  2023-06-01              Number days with snow depth > 1 inch.   
2  1863-

So we have a table that holds the variables that are under the data categories that we are interested in and the description of those variables.

In [230]:
# We get a list with the category ids
datatype_ids = list(variables_df['id'].unique())

There is a limit of 1000 rows that we can query from the database. In order to export the full dataset we need to request batches of 1000 rows, with the next batch to start where the previous batch stopped. In order to identify this point, we need to identify the last date that the dataset describes and start the new export from that date. This way we avoid dealing with dates where the limit was reached before the measurements from all stations are exported. In the final dataset there will be duplicates for those dates and some stations measurements, but we can simply drop them.

In [282]:
dt = pd.DataFrame()
minDate = '2014-01-01'
maxDate = pd.Timestamp.today().strftime('%Y-%m-%d')

for id in countries_select_ids:
    print(id)

    for datatype in datatype_ids:
        print(datatype)
        # Initialize the date and bool params for each of the country
        startDate = minDate
        endDate = maxDate
        finished = False

        # The loop runs while we have not reached the final date or if there is no new data to export
        while ((startDate <= maxDate) & (finished == False)) :
            print(startDate)
            data_parameters = {'datasetid':'GSOM', 'locationcategoryid':'CNTRY', 'locationid':id, 'datatypeid':datatype, 'startdate':startDate, 'enddate':endDate, 'limit':1000}
            xi = requests.get('https://www.ncei.noaa.gov/cdo-web/api/v2/data?', params = data_parameters, 
                    headers=token_data)

            print(xi.url)
            ji=xi.json()
            if (len(ji.keys())==0):
                dti = pd.DataFrame()
            else:
                ji = ji['results']
                dti = pd.DataFrame(ji)
                dti['countryid']=id
                dti['datatype']=datatype

            # Keep only the part of the dataset that is not yet in dt with an anti-join
            if ((dt.empty == False)& (dti.empty == False)) :
                dti = dti.merge(dt[['date', 'station', 'countryid']], on = ['date', 'station', 'countryid'], how = 'left', indicator = True)
                dti = dti[dti['_merge']!="both"]
                dti.drop('_merge', axis = 1, inplace = True)
            
            if  (dti.shape[0] == 0) :
                finished = True

            dt = pd.concat([dt,dti], axis=0)

            # Take the max date of the dataset in order to continue the loop
            SD = max(dt.loc[dt['countryid'] == id, 'date'])
            startDate = datetime.strptime(SD, '%Y-%m-%dT%H:%M:%S')
            startDate = startDate.strftime('%Y-%m-%d')

FIPS:FR
CDSD
2014-01-01
https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid=GSOM&locationcategoryid=CNTRY&locationid=FIPS%3AFR&datatypeid=CDSD&startdate=2014-01-01&enddate=2023-08-02&limit=1000
2015-12-01
https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid=GSOM&locationcategoryid=CNTRY&locationid=FIPS%3AFR&datatypeid=CDSD&startdate=2015-12-01&enddate=2023-08-02&limit=1000
2017-12-01
https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid=GSOM&locationcategoryid=CNTRY&locationid=FIPS%3AFR&datatypeid=CDSD&startdate=2017-12-01&enddate=2023-08-02&limit=1000
2019-11-01
https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid=GSOM&locationcategoryid=CNTRY&locationid=FIPS%3AFR&datatypeid=CDSD&startdate=2019-11-01&enddate=2023-08-02&limit=1000
2022-01-01
https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid=GSOM&locationcategoryid=CNTRY&locationid=FIPS%3AFR&datatypeid=CDSD&startdate=2022-01-01&enddate=2023-08-02&limit=1000
EMNT
2014-01-01
https://www.ncei.noaa.gov/cdo-web/api/v2/data?

Check the shape and the types of the features that are available in the dataset.

In [284]:
dt.shape

(5237, 6)

In [285]:
dt['datatype'].unique()

array(['CDSD', 'EMNT', 'EMXT', 'HDSD', 'TAVG', 'DSND', 'EMXP'],
      dtype=object)

In [66]:
max_dates_country = dt.groupby(['countryid'])[['date']].agg(max)
print(max_dates_country)

                          date
countryid                     
FIPS:FR    2015-10-01T00:00:00
FIPS:GR    2023-06-01T00:00:00


In [68]:
dt[dt['date'].isin(max_dates_country['date'].unique())].sort_values('station')

,date,datatype,station,attributes,value,countryid
968,2015-10-01T00:00:00,TAVG,GHCND:FGM00081401,"4,S",29.10,FIPS:FR
969,2015-10-01T00:00:00,TAVG,GHCND:FR000007130,",E",12.38,FIPS:FR
970,2015-10-01T00:00:00,TAVG,GHCND:FR000007190,",E",10.51,FIPS:FR
971,2015-10-01T00:00:00,TAVG,GHCND:FR000007255,",E",11.36,FIPS:FR
972,2015-10-01T00:00:00,TAVG,GHCND:FR000007510,",E",13.87,FIPS:FR
973,2015-10-01T00:00:00,TAVG,GHCND:FR000007630,",E",14.73,FIPS:FR
974,2015-10-01T00:00:00,TAVG,GHCND:FR000007650,"2,E",15.55,FIPS:FR
975,2015-10-01T00:00:00,TAVG,GHCND:FR000007747,",E",15.93,FIPS:FR
976,2015-10-01T00:00:00,TAVG,GHCND:FR069029001,"2,E",11.66,FIPS:FR
977,2015-10-01T00:00:00,TAVG,GHCND:FRE00104036,",E",11.18,FIPS:FR


Depending on the dates we evaluate the number of data points.
12 data points per year (one for every month) means that we need to consider 120 points per decade. But there are many stations so they multiply with the number of stations.

In [10]:
pd.date_range('2023-07-01',periods=10,freq='M')

DatetimeIndex(['2023-07-31', '2023-08-31', '2023-09-30', '2023-10-31',
               '2023-11-30', '2023-12-31', '2024-01-31', '2024-02-29',
               '2024-03-31', '2024-04-30'],
              dtype='datetime64[ns]', freq='M')

In [223]:
dt['datatype'].unique()


array(['DP01', 'DP10', 'DP1X', 'DYXP', 'EMXP', 'PRCP', 'DT00', 'DT32',
       'DYNT', 'EMNT', 'TMIN', 'DX32', 'DX70', 'DX90', 'DYXT', 'EMXT',
       'TMAX', 'CDSD', 'CLDD', 'HDSD', 'HTDD', 'TAVG'], dtype=object)

We save the dataset to a csv file to start our EDA.

In [290]:
final_dt = dt.merge(variables_df[['id', 'datacategoryid', 'name']], left_on = 'datatype', right_on = 'id', how = 'left')

In [293]:
final_dt.to_csv('data.csv')